In [52]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import log, sqrt, exp
from scipy.stats import norm

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 读取数据
df_sita=pd.read_excel('./data/input/合并_关键词得分汇总.xlsx')
df_rate=pd.read_excel('./data/input/三年期国债收益率.xlsx')
df_total=pd.read_csv('./data/input/EV2&MV&PB&TotalAsset.csv')

#BS函数
def BS(v_0,sigma,T,K,r):
    
    #单利转复利
    R=(1+T*r)**(1/T)-1
    d1 = (log(v_0/K) + (R + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    call_price = v_0 * norm.cdf(d1) - K * exp(-R * T) * norm.cdf(d2)
    #put_price = K * exp(-r * T) * norm.cdf(-d2) - v_0 * norm.cdf(-d1)
    #print(v_0,sigma,T,K,r)
    print('数据资产价格为{},d1为{},N(d1)为{},d2为{},N(d2)为{}'.format(call_price,d1,norm.cdf(d1),d2,norm.cdf(d2)))
    return call_price,K * exp(-R * T),norm.cdf(d1),norm.cdf(d2)


In [53]:
df_sita=pd.read_excel('./data/input/合并_关键词得分汇总.xlsx')
df_sita

,stock_id,stock_name,year,关键词得分(%)_总和,总分词数
0,1,平安银行,2014,0.035355,38311
1,1,平安银行,2015,0.065779,36503
2,1,平安银行,2016,0.054636,34884
3,1,平安银行,2017,0.062643,37719
4,1,平安银行,2018,0.080308,48356
...,...,...,...,...,...
34502,900957,XXXX,2014,0.039470,17534
34503,900957,XXXX,2015,0.032106,18886
34504,900957,XXXX,2017,0.054248,19253
34505,900957,XXXX,2018,0.060726,20179


In [54]:
df_rate['rate_3'].values[-1]

0.0119

In [55]:

#读取sita
sl=df_sita[df_sita['stock_id']==2594]['关键词得分(%)_总和'].values
print(sl)
extended_sl = np.concatenate([
    #np.array([0.04825724, 0.04825724, 0.04825724]),  # 前面添加的3个值
    sl,                            # 原始数组
    np.array([0.040869227])                # 末尾添加的1个值
])
print(extended_sl)
s=df_total.loc[df_total["asharevalue_stat_symbol"] == "002594.SZ"][-11:]["asharevalue_ev2"].values
q=pd.DataFrame(s,columns=['EV2'])
q



[0.04825724 0.04507937 0.04778014 0.04867326 0.04570257 0.03680107
 0.04315211 0.04357814 0.03171501 0.04254683]
[0.04825724 0.04507937 0.04778014 0.04867326 0.04570257 0.03680107
 0.04315211 0.04357814 0.03171501 0.04254683 0.04086923]


,EV2
0,1.066713e+11
1,1.679893e+11
2,1.540533e+11
3,2.189324e+11
4,1.814129e+11
5,1.820087e+11
6,5.572521e+11
7,7.341925e+11
8,6.398386e+11
9,5.443586e+11


In [56]:
a=(df_total.loc[df_total["asharevalue_stat_symbol"] == "002594.SZ"][4:]["asharevalue_ev2"]).pct_change().mean()#["asharevalue_ev2"]

df_total.loc[df_total["asharevalue_stat_symbol"] == "002594.SZ"][4:]
a


0.2983368906197285

In [61]:
def caculate_sita(symbol,yaer_num):
    sita = (df_total.loc[df_total["asharevalue_stat_symbol"] == symbol][-(yaer_num):]["balance_stat_total_assets"]).pct_change().mean()
    return sita


In [59]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import log, sqrt, exp
from scipy.stats import norm
from arima import predict_arima

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 读取数据

#BS函数
def BS(v_0,sigma,T,K,r):
    
    #单利转复利
    R=(1+T*r)**(1/T)-1
    d1 = (log(v_0/K) + (R + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    call_price = v_0 * norm.cdf(d1) - K * exp(-R * T) * norm.cdf(d2)
    #print('数据资产价格为{},d1为{},N(d1)为{},d2为{},N(d2)为{}'.format(call_price,d1,norm.cdf(d1),d2,norm.cdf(d2)))
    return call_price,K * exp(-R * T),norm.cdf(d1),norm.cdf(d2)

def caculate_sigma(symbol,yaer_num):
    sigma = (df_total.loc[df_total["asharevalue_stat_symbol"] == symbol][-(yaer_num):]["balance_stat_total_assets"]).pct_change().mean()
    return sigma

#返回K的时间序列
def caculate_K(symbol,yaer_num):
    return predict_arima(p=5,d=1,q=6,T=3,symbol="002594.SZ",yaer_num=11)


def main_bs(symbol,yaer_num):
    v_0 = df_total.loc[df_total["asharevalue_stat_symbol"] == symbol][-(yaer_num):]["asharevalue_ev2"].values
    sigma = caculate_sigma(symbol,yaer_num)
    K_series = caculate_K(symbol,yaer_num)
    r = df_rate['rate_3'].values
    bs_results = []
    for i in range(yaer_num):
        bs_result = BS(v_0[i],sigma,3,K_series[i+3],r[i])
        bs_results.append(bs_result[0])
    print(v_0[-1])
    return bs_results


In [62]:
a = main_bs(symbol="002594.SZ",yaer_num=10)

FileNotFoundError: [Errno 2] No such file or directory: '合并_关键词得分汇总.xlsx'

In [46]:
df_sig=pd.read_csv('年sig.csv')
df_sig['sig']

0     0.412174
1     0.721133
2     0.326789
3     0.335145
4     0.390771
5     0.335358
6     0.588341
7     0.559738
8     0.438946
9     0.271108
10    0.341313
Name: sig, dtype: float64

In [18]:
df_total.loc[df_total["asharevalue_stat_symbol"] == "002594.SZ"][-11:]#["asharevalue_ev2"]

,Unnamed: 0,asharevalue_stat_symbol,asharevalue_stat_report_date,asharevalue_stat_stat_date,asharevalue_ev2,asharevalue_stat_total_mv,asharevalue_stat_pb_mrq,asharevalue_stat_pb_csi,balance_stat_total_assets
13495,1053,002594.SZ,2015-03-30,2014-12-31,1.066713e+11,8.145926e+10,3.7287,3.73,9.400886e+10
16081,1055,002594.SZ,2016-03-29,2015-12-31,1.679893e+11,1.332225e+11,5.8459,5.50,1.154858e+11
18882,1053,002594.SZ,2017-03-29,2016-12-31,1.540533e+11,1.235117e+11,2.8978,2.68,1.450708e+11
21905,1051,002594.SZ,2018-03-28,2017-12-31,2.189324e+11,1.700318e+11,3.5346,3.28,1.780994e+11
25364,1051,002594.SZ,2019-03-28,2018-12-31,1.814129e+11,1.323561e+11,2.7506,2.55,1.945711e+11
28919,1045,002594.SZ,2020-04-22,2019-12-31,1.820087e+11,1.182755e+11,2.4806,2.29,1.956416e+11
32659,1037,002594.SZ,2021-03-30,2020-12-31,5.572521e+11,5.087781e+11,9.3045,8.87,2.010173e+11
36700,1057,002594.SZ,2022-03-30,2021-12-31,7.341925e+11,7.254733e+11,9.4643,9.30,2.957801e+11
41325,1084,002594.SZ,2023-03-29,2022-12-31,6.398386e+11,6.548274e+11,7.2973,7.30,4.938606e+11
46228,1088,002594.SZ,2024-03-27,2023-12-31,5.443586e+11,5.723367e+11,4.4388,4.44,6.795477e+11


In [ ]:
sigma = (df_total.loc[df_total["asharevalue_stat_symbol"] == "002594.SZ"][-(11):]["balance_stat_total_assets"]).pct_change().mean()
sigma

0.3259864186165659

In [44]:
mask = df_total["asharevalue_stat_symbol"] == "002594.SZ"
last11 = df_total.loc[mask].tail(11)

# 优先使用年份或报告日期
if "stat_year" in last11.columns:
    years = last11["stat_year"].values
elif "asharevalue_stat_reportdate" in last11.columns:
    years = pd.to_datetime(last11["asharevalue_stat_reportdate"]).dt.year.values
else:
    years = np.arange(1, len(a) + 1)

out = pd.DataFrame({'year': years, 'bs_price': a})
out.to_csv('bs_prices_002594_with_year.csv', index=False, encoding='utf-8-sig')

In [64]:
mask = df_total["asharevalue_stat_symbol"] == "002594.SZ"
last11 = df_total.loc[mask].tail(11)
last11

,Unnamed: 0,asharevalue_stat_symbol,asharevalue_stat_report_date,asharevalue_stat_stat_date,asharevalue_ev2,asharevalue_stat_total_mv,asharevalue_stat_pb_mrq,asharevalue_stat_pb_csi,balance_stat_total_assets
13495,1053,002594.SZ,2015-03-30,2014-12-31,1.066713e+11,8.145926e+10,3.7287,3.73,9.400886e+10
16081,1055,002594.SZ,2016-03-29,2015-12-31,1.679893e+11,1.332225e+11,5.8459,5.50,1.154858e+11
18882,1053,002594.SZ,2017-03-29,2016-12-31,1.540533e+11,1.235117e+11,2.8978,2.68,1.450708e+11
21905,1051,002594.SZ,2018-03-28,2017-12-31,2.189324e+11,1.700318e+11,3.5346,3.28,1.780994e+11
25364,1051,002594.SZ,2019-03-28,2018-12-31,1.814129e+11,1.323561e+11,2.7506,2.55,1.945711e+11
28919,1045,002594.SZ,2020-04-22,2019-12-31,1.820087e+11,1.182755e+11,2.4806,2.29,1.956416e+11
32659,1037,002594.SZ,2021-03-30,2020-12-31,5.572521e+11,5.087781e+11,9.3045,8.87,2.010173e+11
36700,1057,002594.SZ,2022-03-30,2021-12-31,7.341925e+11,7.254733e+11,9.4643,9.30,2.957801e+11
41325,1084,002594.SZ,2023-03-29,2022-12-31,6.398386e+11,6.548274e+11,7.2973,7.30,4.938606e+11
46228,1088,002594.SZ,2024-03-27,2023-12-31,5.443586e+11,5.723367e+11,4.4388,4.44,6.795477e+11


In [49]:
# 计算 BS 结果
bs_vals = main_bs(symbol="002594.SZ", yaer_num=11)

# 取对应的 11 期原始数据（用于年份和对比）
mask = df_total["asharevalue_stat_symbol"] == "002594.SZ"
last11 = df_total.loc[mask].tail(11)

# 优先使用年份/日期列；否则用索引编号
if "stat_year" in last11.columns:
    x = last11["stat_year"].values
elif "asharevalue_stat_reportdate" in last11.columns:
    x = pd.to_datetime(last11["asharevalue_stat_reportdate"]).dt.year.values
else:
    x = np.arange(1, len(bs_vals) + 1)

# 可选：叠加原始 EV2 进行对比
ev2_vals = last11["asharevalue_ev2"].values if "asharevalue_ev2" in last11.columns else None

plt.figure(figsize=(10, 6), dpi=600)
plt.plot(x, bs_vals, 'b-o', label='BS 理论价格', linewidth=2, markersize=5)
if ev2_vals is not None:
    plt.plot(x, ev2_vals, 'gray', label='EV2（现值）', linewidth=2)

plt.title('BS 估值（近11期）')
plt.xlabel('Year' if isinstance(x[0], (np.integer, int)) else 'Index')
plt.ylabel('Price')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='best')
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '合并_关键词得分汇总.xlsx'